In [9]:
import cv2
import time
import HandTrackingModule as htm

In [10]:
wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
detector = htm.handDetector()
tipIds = [4, 8, 12, 16, 20]
pTime = 0
paused = False

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)

    if len(lmList) != 0:
        fingers = []   
        if lmList[tipIds[0]][1] > lmList[tipIds[0]-1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
        for id in range(1,5):
            if lmList[tipIds[id]][2] < lmList[tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        #print(fingers)    
        totalFingers  = fingers.count(1)
        cv2.imshow("Image", img)
        # Pause screen if 4 or more fingers are detected and not already paused
        if totalFingers >= 4 and not paused:
            paused = True
            cv2.putText(img, "Paused", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        if cv2.waitKey(1) and totalFingers < 4 and paused:
            paused = False
            break
        else:
            cv2.imshow("Image", img)

    if paused:
        cv2.putText(img, "Paused", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime =cTime

        cv2.putText(img, f'FPS : {int(fps)}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Image", img)
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()